In [8]:
from wilson import Wilson
import flavio
import warnings
import numpy as np
warnings.simplefilter('ignore')

In [2]:
m = flavio.combine_measurements('Rtaul(B->Dlnu)')

/home/jorge/.cache/pypoetry/virtualenvs/banomalies-ml-24-q3wM3yKA-py3.10/lib/python3.10/site-packages/flavio/functions.py:336: UserWarning: 5 of the measurements of 'Rtaul(B->Dlnu)' are multivariate. This can lead to inconsistent results as the other observables are profiled over. To be consistent, you should perform a multivariate combination that is not yet supported by `combine_measurements`.
  warnings.warn(("{} of the measurements of '{}' are multivariate. "


In [3]:
m.get_central()

0.3376994776156502

In [36]:
def chi2(cVL, cVR, cT):
    w = Wilson({'CVL_bctaunutau': cVL, 'CVR_bctaunutau': cVR, 'CT_bctaunutau': cT}, 4.8, 'WET', 'flavio')
    sumchi2 = 0
    m = flavio.combine_measurements('Rtaul(B->Dlnu)')
    sumchi2 += (flavio.np_prediction('Rtaul(B->Dlnu)', w)- m.get_central())**2/m.get_error_left()**2
    m = flavio.combine_measurements('Rtaul(B->D*lnu)')
    sumchi2 += (flavio.np_prediction('Rtaul(B->D*lnu)', w)- m.get_central())**2/m.get_error_left()**2
    m = flavio.combine_measurements('Rtaumu(Bc->J/psilnu)')
    sumchi2 += (flavio.np_prediction('Rtaumu(Bc->J/psilnu)', w)- m.get_central())**2/m.get_error_left()**2
    return sumchi2

chi2v = np.vectorize(chi2)

In [11]:
xcVL, xcVR, xcT = np.meshgrid(np.linspace(0.25, 0.5, 20), np.linspace(-0.7, -0.35, 20), np.linspace(0.2, 0.5, 20))

In [12]:
chi2points = chi2v(xcVL, xcVR, xcT)

In [21]:
np.argmin(chi2points)

6089

In [17]:
chi2points.shape

(20, 20, 20)

In [25]:
xcVL.flatten()[6089]

0.3026315789473684

In [26]:
xcVR.flatten()[6089]

-0.4236842105263158

In [27]:
xcT.flatten()[6089]

0.34210526315789475

In [28]:
from iminuit import Minuit

In [42]:
mm = Minuit(chi2, cVL=0.302, cVR=-0.424, cT=0.342)

In [43]:
mm.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 0.7336                     │              Nfcn = 43               │
│ EDM = 2.26e-05 (Goal: 0.0002)    │            time = 1.1 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ cVL  │   0.30    │   0.14    │            │            │         │         │       │
│ 1 │ cVR  │   -0.42   │   0.10    │            │            │         │         │       │
│ 2 │ cT   │    0.3    │    0.4    │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────┬─────────────────────────┐
│     │     cVL     cVR      cT │
├─────┼─────────────────────────┤
│ cVL │  0.0195   0.012  -0.050 │
│ cVR │   0.012 0.00903  -0.032 │
│  cT │  -0.050  -0.032   0.131 │
└─────┴─────────────────────────┘

In [44]:
mm.values

<ValueView cVL=0.30186688851235455 cVR=-0.42287833212552134 cT=0.34278773288887743>

In [45]:
def chi2_cs(cVL, cVR, cT, cSL, cSR):
    w = Wilson({'CVL_bctaunutau': cVL, 'CVR_bctaunutau': cVR, 'CT_bctaunutau': cT, 'CSL_bctaunutau': cSL, 'CSR_bctaunutau': cSR}, 4.8, 'WET', 'flavio')
    sumchi2 = 0
    m = flavio.combine_measurements('Rtaul(B->Dlnu)')
    sumchi2 += (flavio.np_prediction('Rtaul(B->Dlnu)', w)- m.get_central())**2/m.get_error_left()**2
    m = flavio.combine_measurements('Rtaul(B->D*lnu)')
    sumchi2 += (flavio.np_prediction('Rtaul(B->D*lnu)', w)- m.get_central())**2/m.get_error_left()**2
    m = flavio.combine_measurements('Rtaumu(Bc->J/psilnu)')
    sumchi2 += (flavio.np_prediction('Rtaumu(Bc->J/psilnu)', w)- m.get_central())**2/m.get_error_left()**2
    return sumchi2

In [73]:
mm2 = Minuit(chi2_cs, cVL=0.302, cVR=-0.424, cT=0.342, cSL=0.0, cSR=0.0)

In [74]:
mm2.migrad(ncall=10000, iterate=50)

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 0.4148                     │             Nfcn = 3643              │
│ EDM = 1.82e+07 (Goal: 0.0002)    │           time = 85.0 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│         INVALID Minimum          │   ABOVE EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ cVL  │  -0.319   │   0.035   │            │            │         │         │       │
│ 1 │ cVR  │  -1.723   │   0.034   │            │            │         │         │       │
│ 2 │ cT   │480.8428e-3│ 0.0007e-3 │            │            │         │         │       │
│ 3 │ cSL  │   2.36    │   0.07    │            │            │         │         │       │
│ 4 │ cSR  │  -0.697   │   0.008   │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────┬───────────────────────────────────────────────────┐
│     │       cVL       cVR        cT       cSL       cSR │
├─────┼───────────────────────────────────────────────────┤
│ cVL │   0.00124    0.0008 -22.4e-12   -0.0025   0.22e-3 │
│ cVR │    0.0008   0.00115 -26.5e-12   -0.0021   0.02e-3 │
│  cT │ -22.4e-12 -26.5e-12  4.61e-13  54.8e-12  -1.9e-12 │
│ cSL │   -0.0025   -0.0021  54.8e-12   0.00535  -0.35e-3 │
│ cSR │   0.22e-3   0.02e-3  -1.9e-12  -0.35e-3   5.8e-05 │
└─────┴───────────────────────────────────────────────────┘

In [68]:
mm2.values

<ValueView cVL=0.302 cVR=-0.424 cT=0.342 cSL=0.0 cSR=0.0>

In [52]:
mm.fval

0.7335676169335652

In [53]:
def chi2_cs_RD(cVL, cVR, cT, cSL, cSR):
    w = Wilson({'CVL_bctaunutau': cVL, 'CVR_bctaunutau': cVR, 'CT_bctaunutau': cT, 'CSL_bctaunutau': cSL, 'CSR_bctaunutau': cSR}, 4.8, 'WET', 'flavio')
    sumchi2 = 0
    m = flavio.combine_measurements('Rtaul(B->Dlnu)')
    sumchi2 += (flavio.np_prediction('Rtaul(B->Dlnu)', w)- m.get_central())**2/m.get_error_left()**2
    #m = flavio.combine_measurements('Rtaul(B->D*lnu)')
    #sumchi2 += (flavio.np_prediction('Rtaul(B->D*lnu)', w)- m.get_central())**2/m.get_error_left()**2
    #m = flavio.combine_measurements('Rtaumu(Bc->J/psilnu)')
    #sumchi2 += (flavio.np_prediction('Rtaumu(Bc->J/psilnu)', w)- m.get_central())**2/m.get_error_left()**2
    return sumchi2

In [57]:
mmRD = Minuit(chi2_cs_RD, cVL=0.302, cVR=-0.424, cT=0.342, cSL=0.0, cSR=0.0)

In [58]:
mmRD.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 3.486e-08                  │              Nfcn = 62               │
│ EDM = 3.48e-08 (Goal: 0.0002)    │            time = 0.5 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │     Covariance FORCED pos. def.      │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ cVL  │    0.3    │    0.5    │            │            │         │         │       │
│ 1 │ cVR  │   -0.4    │    0.5    │            │            │         │         │       │
│ 2 │ cT   │    0.3    │    0.9    │            │            │         │         │       │
│ 3 │ cSL  │    0.0    │    0.8    │            │            │         │         │       │
│ 4 │ cSR  │    0.0    │    0.8    │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────┬───────────────────────────────┐
│     │   cVL   cVR    cT   cSL   cSR │
├─────┼───────────────────────────────┤
│ cVL │ 0.299 -0.07 -0.12 -0.11 -0.11 │
│ cVR │ -0.07 0.299 -0.12 -0.11 -0.11 │
│  cT │ -0.12 -0.12 0.773  -0.2  -0.2 │
│ cSL │ -0.11 -0.11  -0.2 0.693  -0.2 │
│ cSR │ -0.11 -0.11  -0.2  -0.2 0.693 │
└─────┴───────────────────────────────┘

In [56]:
def chi2_cs_RDs(cVL, cVR, cT, cSL, cSR):
    w = Wilson({'CVL_bctaunutau': cVL, 'CVR_bctaunutau': cVR, 'CT_bctaunutau': cT, 'CSL_bctaunutau': cSL, 'CSR_bctaunutau': cSR}, 4.8, 'WET', 'flavio')
    sumchi2 = 0
    #m = flavio.combine_measurements('Rtaul(B->Dlnu)')
    #sumchi2 += (flavio.np_prediction('Rtaul(B->Dlnu)', w)- m.get_central())**2/m.get_error_left()**2
    m = flavio.combine_measurements('Rtaul(B->D*lnu)')
    sumchi2 += (flavio.np_prediction('Rtaul(B->D*lnu)', w)- m.get_central())**2/m.get_error_left()**2
    #m = flavio.combine_measurements('Rtaumu(Bc->J/psilnu)')
    #sumchi2 += (flavio.np_prediction('Rtaumu(Bc->J/psilnu)', w)- m.get_central())**2/m.get_error_left()**2
    return sumchi2

In [59]:
mmRDs = Minuit(chi2_cs_RDs, cVL=0.302, cVR=-0.424, cT=0.342, cSL=0.0, cSR=0.0)

In [60]:
mmRDs.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 5.903e-06                  │              Nfcn = 66               │
│ EDM = 5.89e-06 (Goal: 0.0002)    │            time = 0.7 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │     Covariance FORCED pos. def.      │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ cVL  │   0.30    │   0.17    │            │            │         │         │       │
│ 1 │ cVR  │   -0.42   │   0.28    │            │            │         │         │       │
│ 2 │ cT   │    0.3    │    2.7    │            │            │         │         │       │
│ 3 │ cSL  │     0     │     1     │            │            │         │         │       │
│ 4 │ cSR  │     0     │     1     │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────┬────────────────────────────────────┐
│     │    cVL    cVR     cT    cSL    cSR │
├─────┼────────────────────────────────────┤
│ cVL │ 0.0292 -0.017 -0.321 -0.050  0.050 │
│ cVR │ -0.017 0.0794   0.56   0.09  -0.09 │
│  cT │ -0.321   0.56   7.36    1.8   -1.9 │
│ cSL │ -0.050   0.09    1.8   1.05   -0.3 │
│ cSR │  0.050  -0.09   -1.9   -0.3   1.05 │
└─────┴────────────────────────────────────┘

In [61]:
def chi2_cs_RJpsi(cVL, cVR, cT, cSL, cSR):
    w = Wilson({'CVL_bctaunutau': cVL, 'CVR_bctaunutau': cVR, 'CT_bctaunutau': cT, 'CSL_bctaunutau': cSL, 'CSR_bctaunutau': cSR}, 4.8, 'WET', 'flavio')
    sumchi2 = 0
    m = flavio.combine_measurements('Rtaul(B->Dlnu)')
    sumchi2 += (flavio.np_prediction('Rtaul(B->Dlnu)', w)- m.get_central())**2/m.get_error_left()**2
    #m = flavio.combine_measurements('Rtaul(B->D*lnu)')
    #sumchi2 += (flavio.np_prediction('Rtaul(B->D*lnu)', w)- m.get_central())**2/m.get_error_left()**2
    m = flavio.combine_measurements('Rtaumu(Bc->J/psilnu)')
    sumchi2 += (flavio.np_prediction('Rtaumu(Bc->J/psilnu)', w)- m.get_central())**2/m.get_error_left()**2
    return sumchi2

In [62]:
mmRJpsi = Minuit(chi2_cs_RJpsi, cVL=0.302, cVR=-0.424, cT=0.342, cSL=0.0, cSR=0.0)

In [63]:
mmRJpsi.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 1.789e-06                  │              Nfcn = 102              │
│ EDM = 1.77e-06 (Goal: 0.0002)    │            time = 1.6 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │     Covariance FORCED pos. def.      │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ cVL  │    0.5    │    0.4    │            │            │         │         │       │
│ 1 │ cVR  │   -0.6    │    0.4    │            │            │         │         │       │
│ 2 │ cT   │    0.4    │    0.8    │            │            │         │         │       │
│ 3 │ cSL  │   -0.1    │    1.0    │            │            │         │         │       │
│ 4 │ cSR  │     0     │     1     │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────┬───────────────────────────────┐
│     │   cVL   cVR    cT   cSL   cSR │
├─────┼───────────────────────────────┤
│ cVL │ 0.178  0.06 -0.19 -0.03 -0.15 │
│ cVR │  0.06 0.184 -0.05 -0.24 -0.12 │
│  cT │ -0.19 -0.05 0.586  -0.1  -0.2 │
│ cSL │ -0.03 -0.24  -0.1 0.904  -0.3 │
│ cSR │ -0.15 -0.12  -0.2  -0.3 0.989 │
└─────┴───────────────────────────────┘